# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [29]:
import json

def general_score(age, temperature, pH, heart_rate, respiratory_rate, sodium, 
              potassium, hematocrit, white_blood_count, config_file):
    
    general_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    age_rules = config.get("Age")
    temperature_rules = config.get("Temperature")
    pH_rules = config.get("pH")
    heart_rate_rules = config.get("Heart Rate")
    respiratory_rate_rules = config.get("Respiratory Rate")
    sodium_rules = config.get("Sodium")
    potassium_rules = config.get("Potassium")
    hematocrit_rules = config.get("Hematocrit")
    white_blood_count_rules = config.get("White Blood Count")
    
    for rule in age_rules:
        if age >= rule.get('min') and age < rule.get('max'):
            age_score = rule.get('points')
    for rule in temperature_rules:
        if temperature >= rule.get('min') and temperature < rule.get('max'):
            temperature_score = rule.get('points')
    for rule in pH_rules:
        if pH >= rule.get('min') and pH < rule.get('max'):
            pH_score = rule.get('points')
    for rule in heart_rate_rules:
        if heart_rate >= rule.get('min') and heart_rate < rule.get('max'):
            heart_rate_score = rule.get('points')
    for rule in respiratory_rate_rules:
        if respiratory_rate >= rule.get('min') and respiratory_rate < rule.get('max'):
            respiratory_rate_score = rule.get('points')
    for rule in sodium_rules:
        if sodium >= rule.get('min') and sodium < rule.get('max'):
            sodium_score = rule.get('points')
    for rule in potassium_rules:
        if potassium >= rule.get('min') and potassium < rule.get('max'):
            potassium_score = rule.get('points')
    for rule in hematocrit_rules:
        if hematocrit >= rule.get('min') and hematocrit < rule.get('max'):
            hematocrit_score = rule.get('points')
    for rule in white_blood_count_rules:
        if white_blood_count >= rule.get('min') and white_blood_count < rule.get('max'):
            white_blood_count_score = rule.get('points')
    
    ### END SOLUTION

    return general_score

In [59]:
GENERAL_CONFIG_FILE = "apache.json"

In [41]:
def organ_failure_score(organ_failure, config_file):
    
    organ_failure_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    organ_failure_rules = config.get("Organ Failure History")
    
    for rule in organ_failure_rules:
        if "Nonoperative":
            return 5
        elif "Emergency":
            return 5
        elif "Elective":
            return 2
        elif "None":
            return 0
    
    ### END SOLUTION

    return organ_failure_score

In [26]:
ORGANFAILURE_CONFIG_FILE = "apache.json"

In [57]:
import json

def fio2_score(pao2, aa_gradient, config_file):
    
    fio2_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    fio2_rules = config.get("FiO2")
    
    for rule in fio2_rules:
        if pao2 >= rule.get('min') and pao2 < rule.get('max'):
            fio2_score = rule.get('points')
    for rule in fio2_rules:
        if aa_gradient >= rule.get('min') and aa_gradient < rule.get('max'):
            fio2_score = rule.get('points')
    
    ### END SOLUTION

    return fio2_score

In [49]:
FIO2_CONFIG_FILE = "apache.json"

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [55]:
assert organ_failure_score("Nonoperative", ORGANFAILURE_CONFIG_FILE) == 5
assert organ_failure_score("Emergency", ORGANFAILURE_CONFIG_FILE) == 5

In [107]:
assert fio2_score(72,199,ORGANFAILURE_CONFIG_FILE) == 0

AttributeError: 'str' object has no attribute 'get'

In [62]:
assert general_score(46, 33, 7.26, 56, 7, 121, 2.6, 3.1, 21, 2, GENERAL_CONFIG_FILE = "apache.json") == 2

TypeError: general_score() got an unexpected keyword argument 'GENERAL_CONFIG_FILE'

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [89]:
import json
def apache_score(organ_failure, age, creatinine, temperature, pH,
                 heart_rate, respiratory_rate, sodium, potassium,
                 acute_renal_failure, hematocrit, white_blood_count, glasgow_scale,
                 fio2, pao2, aa_gradient, config_file):

    range_loop_parameters = [
        "Age",
        "Temperature",
        "pH",
        "Heart Rate",
        "Respiratory Rate",
        "Sodium",
        "Potassium",
        "Hematocrit",
        "White Blood Count",
    ]
    
    value_dict = {
        "Organ Failure History": organ_failure,
        "Age": age,
        "Temperature": temperature,
        "pH": pH,
        "Heart Rate": heart_rate,
        "Respiratory Rate": respiratory_rate,
        "Sodium": sodium,
        "Potassium": potassium,
        "Hematocrit": hematocrit,
        "White Blood Count": white_blood_count,
        "Creatinine": creatinine 
    }
    
    value = value_dict['Creatinine']
    if acute_renal_failure == 'Acute Renal Failure' :
        list_of_ranges = config['Creatinine']['Acute Renal Failure']
        print(get_score(value, list_of_ranges), 'for', 'acute renal failure')
        score += get_score(value, list_of_ranges)
    elif acute_renal_failure == 'Chronic Renal Failure':
        list_of_ranges = config['Creatinine']['Chronic Renal Failure']
        print(get_score(value, list_of_ranges), 'for', 'chronic renal failure')
        score += get_score(value, list_of_ranges)
    else:
        return None 
        
    config = json.load(open(config_file))
    for parameter in range_loop_parameters:
        value = value_dict[param]
        print(get_score(value, list_of_ranges), 'for', parameters)
        score += get_score(value, list_of_ranges)
        
    if fio2 < 50:
        value = value_dict['PaO2']
        list_of_ranges = config['FiO2']['PaO2']
        print(get_score(value, list_of_ranges), 'for', 'PaO2')
        score += get_score(value, list_of_ranges)
    else:
        value = value_dict['A-a gradient']
        list_of_ranges = config['FiO2']['A-a gradient']
        print(get_score(value, list_of_ranges), 'for', 'a-a gradient')
        score += get_score(value, list_of_ranges)
        
    print(config['Organ Failure History'][organ_failure], 'for', 'organ failure')
    score += config['Organ Failure History'][organ_failure]
        
    

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [92]:
assert apache_score(organ_failure = 'Emergency', creatinine = 1.1, age = 66, temperature = 38, pH = 7.66, 
                    heart_rate =  120, respiratory_rate = 11, sodium = 165, potassium = 3.2, hematocrit = 40, 
                    white_blood_count = 16, glasgow_scale = 12, fio2 = 50, pao2 = 54, aa_gradient = 220, 
                    acute_renal_failure = 1.6, config_file = 'apache.json') == 36

AssertionError: 

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [93]:
def patient_csv(url, sep=','):
    
    value = {
        "Organ Failure History": "organ_failure",
        "Age": "age",
        "Temperature": "temperature",
        "pH": "pH",
        "Heart Rate": "heart_rate",
        "Respiratory Rate": "respiratory_rate",
        "Sodium": "sodium",
        "Potassium": "potassium",
        "Hematocrit": "hematocrit",
        "White Blood Count": "white_blood_count",
        "FiO2": "fio2",
        "A-a gradient": "aa_gradient",
        "PaO2": "pao2",
        "Acute Renal Failure": "acute_renal_failure",
        "Glasgow Coma Scale": "glasgow_scale"
    }
    
    df = pd.read.csv(url, sep=sep)
    
    df = df.rename(columns=value_dict)
    
    scores = []
    for row, patient in df.drop(columns=["Patient"]).iterrows():
        patient = patient.to_dict()
        scores.append(apache_score(**patient, config_file = "apache.json"))
        print(patient)
        
    return scores

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [105]:
sample = 'apache.json'

In [97]:
import pandas as pd
df = pd.read_csv("https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")

In [98]:
df_2 = pd.read_csv("https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")

In [104]:
df_test = pd.read_csv("https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv")

In [106]:
df_test["sample"] = df

ValueError: Wrong number of items passed 17, placement implies 1

In [103]:
(df_test["total"] == df_test["sample"]).sum()

KeyError: 'total'